In [1]:
# Verificar que estamos usando el GPU de la laptop:
!pip install torch
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tensor = torch.randn(2, 2).to(device)
print(tensor.device)

print(torch.cuda.device_count())  # Imprime el número de GPUs disponibles

cpu
0


In [3]:
# Instalar Kaggle
!pip install -q kaggle

# Ruta donde se encuentra tu archivo kaggle.json
kaggle_json_path = r'C:\Users\User\Documents\kaggle.json'


In [1]:
import os  # For path manipulation
import shutil  # For file movement (more portable)

# Instalar Kaggle
!pip install -q kaggle

# Ruta donde se encuentra tu archivo kaggle.json
kaggle_json_path = r'C:\Users\User\Documents\kaggle.json'

# Crear el directorio .kaggle si no existe
if not os.path.exists(os.path.expanduser('~/.kaggle')):
    os.makedirs(os.path.expanduser('~/.kaggle'))

# Mover kaggle.json al directorio .kaggle
dest_path = os.path.join(os.path.expanduser('~/.kaggle'), 'kaggle.json')
shutil.move(kaggle_json_path, dest_path)

# Asegurarse de que el archivo kaggle.json tiene permisos correctos
os.chmod(dest_path, 0o600)

# Descargar el dataset CIFAR-10 desde Kaggle
!kaggle datasets download gopalbhattrai/pascal-voc-2012-dataset

Dataset URL: https://www.kaggle.com/datasets/gopalbhattrai/pascal-voc-2012-dataset
License(s): unknown




  0%|          | 0.00/3.52G [00:00<?, ?B/s]
  0%|          | 1.00M/3.52G [00:00<21:32, 2.92MB/s]
  0%|          | 3.00M/3.52G [00:00<12:11, 5.16MB/s]
  0%|          | 5.00M/3.52G [00:00<10:43, 5.87MB/s]
  0%|          | 6.00M/3.52G [00:01<09:37, 6.53MB/s]
  0%|          | 7.00M/3.52G [00:01<08:43, 7.20MB/s]
  0%|          | 9.00M/3.52G [00:01<07:37, 8.25MB/s]
  0%|          | 11.0M/3.52G [00:01<06:51, 9.17MB/s]
  0%|          | 13.0M/3.52G [00:01<05:35, 11.2MB/s]
  0%|          | 15.0M/3.52G [00:01<05:27, 11.5MB/s]
  0%|          | 17.0M/3.52G [00:01<04:54, 12.8MB/s]
  1%|          | 19.0M/3.52G [00:02<05:16, 11.9MB/s]
  1%|          | 21.0M/3.52G [00:02<05:35, 11.2MB/s]
  1%|          | 24.0M/3.52G [00:02<04:21, 14.3MB/s]
  1%|          | 26.0M/3.52G [00:02<04:07, 15.2MB/s]
  1%|          | 28.0M/3.52G [00:02<03:52, 16.1MB/s]
  1%|          | 30.0M/3.52G [00:02<04:13, 14.8MB/s]
  1%|          | 33.0M/3.52G [00:03<03:22, 18.5MB/s]
  1%|          | 36.0M/3.52G [00:03<03:36, 17.3MB/s]
 

In [2]:
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from PIL import Image

def get_segment_points(mask_path, class_id, img_width, img_height):
    try:
        # Lee la máscara de segmentación
        mask = np.array(Image.open(mask_path))
        
        # Encuentra los contornos de la clase específica
        binary_mask = (mask == class_id).astype(np.uint8)
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Si no hay contornos, retorna None
        if not contours:
            return None
        
        # Toma el contorno más grande
        contour = max(contours, key=cv2.contourArea)
        
        # Simplifica el contorno para reducir puntos
        epsilon = 0.005 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Convierte los puntos a formato YOLO (normalizado)
        points = []
        for point in approx:
            x = float(point[0][0]) / img_width
            y = float(point[0][1]) / img_height
            points.extend([x, y])
        
        return points
    except Exception as e:
        print(f"Error procesando máscara {mask_path}: {str(e)}")
        return None

def parse_xml_to_yolo_seg(xml_file, mask_dir, class_mapping, output_dir):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        img_width = int(root.find('size/width').text)
        img_height = int(root.find('size/height').text)
        
        # Obtén el nombre base del archivo
        base_name = os.path.splitext(os.path.basename(xml_file))[0]
        
        # Ruta a la máscara correspondiente
        mask_path = os.path.join(mask_dir, f"{base_name}.png")
        
        # Verifica si existe la máscara
        if not os.path.exists(mask_path):
            print(f"No se encontró máscara para {base_name}")
            return False
        
        yolo_annotations = []
        
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            class_id = class_mapping.get(class_name, -1)
            
            if class_id == -1:
                continue
            
            # Obtiene los puntos del segmento
            points = get_segment_points(mask_path, class_id + 1, img_width, img_height)
            
            if points:
                # Formato YOLO para segmentación: class_id x1 y1 x2 y2 ...
                points_str = " ".join([f"{p:.6f}" for p in points])
                yolo_annotations.append(f"{class_id} {points_str}")
        
        # Solo guarda si se encontraron anotaciones
        if yolo_annotations:
            output_path = os.path.join(output_dir, f"{base_name}.txt")
            with open(output_path, "w") as f:
                for annotation in yolo_annotations:
                    f.write(annotation + "\n")
            return True
        else:
            print(f"No se encontraron anotaciones válidas para {base_name}")
            return False
            
    except Exception as e:
        print(f"Error procesando {xml_file}: {str(e)}")
        return False

def process_annotations_folder(input_dir, mask_dir, output_dir, class_mapping):
    os.makedirs(output_dir, exist_ok=True)
    
    # Contadores para estadísticas
    total_files = 0
    processed_files = 0
    skipped_files = 0
    
    # Obtén la lista de archivos XML
    xml_files = [f for f in os.listdir(input_dir) if f.endswith('.xml')]
    
    # Obtén la lista de máscaras
    mask_files = set(os.path.splitext(f)[0] for f in os.listdir(mask_dir) if f.endswith('.png'))
    
    print("Iniciando procesamiento...")
    
    for xml_file in xml_files:
        total_files += 1
        base_name = os.path.splitext(xml_file)[0]
        
        # Verifica si existe la máscara correspondiente
        if base_name not in mask_files:
            print(f"Saltando {xml_file} - No se encontró máscara correspondiente")
            skipped_files += 1
            continue
        
        full_path = os.path.join(input_dir, xml_file)
        if parse_xml_to_yolo_seg(full_path, mask_dir, class_mapping, output_dir):
            processed_files += 1
            print(f"Procesado exitosamente: {xml_file}")
        else:
            skipped_files += 1
    
    # Imprime estadísticas
    print("\nEstadísticas de procesamiento:")
    print(f"Total de archivos XML encontrados: {total_files}")
    print(f"Archivos procesados exitosamente: {processed_files}")
    print(f"Archivos saltados: {skipped_files}")
    print(f"Porcentaje de éxito: {(processed_files/total_files)*100:.2f}%")

# Tu mapeo de clases
class_mapping = {
    "aeroplane": 0,
    "bicycle": 1,
    "bird": 2,
    "boat": 3,
    "bottle": 4,
    "bus": 5,
    "car": 6,
    "cat": 7,
    "chair": 8,
    "cow": 9,
    "diningtable": 10,
    "dog": 11,
    "horse": 12,
    "motorbike": 13,
    "person": 14,
    "pottedplant": 15,
    "sheep": 16,
    "sofa": 17,
    "train": 18,
    "tvmonitor": 19
}

# Rutas
input_dir = r"C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\pascal-voc-2012-dataset\VOC2012_train_val\VOC2012_train_val\Annotations"
mask_dir = r"C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\pascal-voc-2012-dataset\VOC2012_train_val\VOC2012_train_val\SegmentationClass"
output_dir = r"C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset"

# Ejecuta el procesamiento
process_annotations_folder(input_dir, mask_dir, output_dir, class_mapping)

Iniciando procesamiento...
Saltando 2007_000027.xml - No se encontró máscara correspondiente
Procesado exitosamente: 2007_000032.xml
Procesado exitosamente: 2007_000033.xml
Procesado exitosamente: 2007_000039.xml
Procesado exitosamente: 2007_000042.xml
Procesado exitosamente: 2007_000061.xml
Procesado exitosamente: 2007_000063.xml
Procesado exitosamente: 2007_000068.xml
Procesado exitosamente: 2007_000121.xml
Procesado exitosamente: 2007_000123.xml
Procesado exitosamente: 2007_000129.xml
Procesado exitosamente: 2007_000170.xml
Procesado exitosamente: 2007_000175.xml
Procesado exitosamente: 2007_000187.xml
Procesado exitosamente: 2007_000241.xml
Procesado exitosamente: 2007_000243.xml
Procesado exitosamente: 2007_000250.xml
Procesado exitosamente: 2007_000256.xml
Saltando 2007_000272.xml - No se encontró máscara correspondiente
Procesado exitosamente: 2007_000323.xml
Procesado exitosamente: 2007_000332.xml
Procesado exitosamente: 2007_000333.xml
Procesado exitosamente: 2007_000346.xml
P

In [3]:
import os

# Directorios de imágenes y etiquetas
train_images_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\images'
train_labels_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\labels'


# Función para obtener la lista de archivos sin la extensión
def obtener_archivos_sin_extension(directorio, extension):
    archivos = [os.path.splitext(f)[0] for f in os.listdir(directorio) if f.endswith(extension)]
    return set(archivos)

# Obtener los archivos de imágenes y etiquetas (sin extensión)
train_images = obtener_archivos_sin_extension(train_images_dir, '.jpg')
train_labels = obtener_archivos_sin_extension(train_labels_dir, '.txt')


# Comparar imágenes y etiquetas en train y val
imagenes_sin_etiquetas_train = train_images - train_labels


# Mostrar imágenes sin etiquetas sin eliminarlas
print(f'Imágenes sin etiquetas en train: {len(imagenes_sin_etiquetas_train)}')


# Eliminar imágenes sin etiquetas en train
if imagenes_sin_etiquetas_train:
    print('Eliminando imágenes sin etiquetas en train:')
    for img in imagenes_sin_etiquetas_train:
        img_path = os.path.join(train_images_dir, img + '.jpg')
        print(f'Eliminando {img_path}')
        os.remove(img_path)  # Eliminar la imagen


Imágenes sin etiquetas en train: 14212
Eliminando imágenes sin etiquetas en train:
Eliminando C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\images\2011_003194.jpg
Eliminando C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\images\2010_000564.jpg
Eliminando C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\images\2008_001812.jpg
Eliminando C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\images\2009_002443.jpg
Eliminando C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\images\2008_006641.jpg
Eliminando C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\images\2009_004868.jpg
Eliminando C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\images\2008_007353.jpg
Eliminando C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENE

In [4]:
import os
import random
import shutil
from pathlib import Path

# Directorios originales
train_images_dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\images')
train_labels_dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\labels')

# Directorios de validación
val_images_dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\val\images')
val_labels_dir = Path(r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\PASCAL\dataset\val\labels')

# Crear directorios de validación si no existen
val_images_dir.mkdir(exist_ok=True, parents=True)
val_labels_dir.mkdir(exist_ok=True, parents=True)

# Obtener la lista de imágenes y etiquetas
train_images = list(train_images_dir.glob('*.jpg'))  # Asumiendo que las imágenes son .jpg
train_labels = [train_labels_dir / (img.stem + '.txt') for img in train_images]

# Comprobar que la cantidad de imágenes y etiquetas coincidan
assert len(train_images) == len(train_labels), "La cantidad de imágenes y etiquetas no coincide."

# Calcular el 20% del total de imágenes
num_val = int(len(train_images) * 0.2)

# Seleccionar aleatoriamente el 20% de las imágenes y etiquetas
selected_indices = random.sample(range(len(train_images)), num_val)

print(f"Total de archivos encontrados: {len(train_images)}")
print(f"Moviendo {num_val} archivos a validación...")

# Mover las imágenes y etiquetas seleccionadas a las carpetas de validación
for idx in selected_indices:
    # Mover imagen
    shutil.move(str(train_images[idx]), str(val_images_dir / train_images[idx].name))
    # Mover etiqueta
    shutil.move(str(train_labels[idx]), str(val_labels_dir / train_labels[idx].name))

# Contar archivos después del movimiento
remaining_train = len(list(train_images_dir.glob('*.jpg')))
moved_val = len(list(val_images_dir.glob('*.jpg')))

print("\nResumen:")
print(f"Archivos movidos a validación: {moved_val}")
print(f"Archivos restantes en entrenamiento: {remaining_train}")
print(f"Porcentaje en validación: {(moved_val/(moved_val+remaining_train))*100:.1f}%")

Total de archivos encontrados: 2913
Moviendo 582 archivos a validación...

Resumen:
Archivos movidos a validación: 582
Archivos restantes en entrenamiento: 2331
Porcentaje en validación: 20.0%


In [ ]:
import os

# Directorio de las imágenes
images_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\COCO\dataset\images\train'

# Directorio donde se guardará el archivo train2017.txt
output_dir = r'C:\Users\User\Documents\JUPYTER_LOCAL\IMAGENES_ENTRENAMIENTO\SEGMENTACION\COCO\dataset'

# Nombre del archivo de salida
output_file = 'train.txt'

# Función para obtener las rutas de las imágenes
def get_image_paths(images_dir):
    image_paths = []
    for image_file in os.listdir(images_dir):
        if image_file.endswith('.jpg'):
            # Construimos la ruta absoluta de la imagen
            image_path = os.path.join(images_dir, image_file)
            # Convertimos la ruta a formato adecuado para evitar errores de barras invertidas
            image_path = image_path.replace('\\', '/')
            image_paths.append(image_path)
    return image_paths

# Obtener las rutas de las imágenes
image_paths = get_image_paths(images_dir)

# Escribir las rutas en el archivo train2017.txt
output_path = os.path.join(output_dir, output_file)
with open(output_path, 'w') as f:
    for path in image_paths:
        f.write(path + '\n')

print(f"Se ha creado el archivo {output_file} con {len(image_paths)} rutas de imágenes.")

In [1]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tensor = torch.randn(2, 2).to(device)
print(tensor.device)
device = torch.device('cuda:0')  # Forzar uso de la primera GPU NVIDIA

cuda:0


In [2]:
from ultralytics import YOLO
import torch

# Verificar si la GPU está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Cargar el modelo YOLOv8 para detección
model = YOLO('yolo11s-seg.pt')

# Mover el modelo a la GPU si está disponible
model = model.to(device)

# Imprimir el dispositivo utilizado (cuda o cpu)
print("Usando el dispositivo:", device)

# Entrenar el modelo en la GPU
model.train(data=r'VOC_FINAL.yaml', epochs=30, imgsz=640, workers=2)

100%|██████████| 19.7M/19.7M [00:02<00:00, 7.89MB/s]


Usando el dispositivo: cuda
New https://pypi.org/project/ultralytics/8.3.29 available  Update with 'pip install -U ultralytics'
engine\trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=VOC_FINAL.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=2, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/luisangelolveraolvera/general/a6512e45550e4d09b07d98dc55b8872b

COMET INFO: Couldn't find a Git repository in 'C:\\Users\\User\\Documents\\JUPYTER_LOCAL\\IMAGENES_ENTRENAMIENTO\\SEGMENTACION\\PASCAL' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...


100%|██████████| 5.35M/5.35M [00:00<00:00, 10.1MB/s]


...


30 epochs completed in 0.536 hours.
Optimizer stripped from runs\segment\train\weights\last.pt, 20.5MB
Optimizer stripped from runs\segment\train\weights\best.pt, 20.5MB

Validating runs\segment\train\weights\best.pt...
Ultralytics 8.3.13  Python-3.12.7 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11s-seg summary (fused): 265 layers, 10,074,556 parameters, 0 gradients, 35.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  2.95it/s]


                   all        582        861      0.715      0.712      0.743      0.595      0.703      0.694       0.72      0.513
             aeroplane         33         33      0.913      0.955      0.986      0.878      0.913      0.954      0.986      0.727
               bicycle         35         35      0.719      0.743       0.81      0.584      0.529      0.543       0.48       0.18
                  bird         40         40       0.72      0.772      0.797      0.631      0.741      0.786      0.829      0.573
                  boat         26         26      0.778      0.674      0.725      0.508      0.732      0.631      0.656      0.416
                bottle         25         25      0.577       0.48      0.545       0.42      0.582       0.48      0.545      0.422
                   bus         31         31      0.806      0.805      0.912      0.821      0.806      0.803      0.912      0.796
                   car         59         59      0.781      0.665   

COMET ERROR: We failed to read file runs\segment\train\F1_curve.png for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file runs\segment\train\P_curve.png for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file runs\segment\train\R_curve.png for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file runs\segment\train\PR_curve.png for uploading.
Please double-check the file path and permissions
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : thundering_expenditure_3441
COMET INFO:     url                   : https://www.comet.com/luisangelolveraolvera/general/a6512e45

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021847561220>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.03503